In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas as pd
from scipy.stats import norm

In [ ]:
class SABRParameters:
  def __init__(self, alpha:float=0.2, beta:float=0.5,
               rho:float=-0.3, sigma:float=0.25):
    assert alpha > 0, "alpha must be greater than 0"
    assert beta > 0, "beta must be greater than 0"
    assert rho >= -1 and rho <= 1, "rho must be between -1 and 1"
    assert sigma > 0, "sigma must be greater than 0"
    self.alpha = alpha
    self.beta = beta
    self.rho = rho
    self.sigma = sigma

In [ ]:
def sabr_impl_vol_(
      F:float, K:float, T:float, alpha:float,
      beta:float, rho:float, sigma0:float
    ):
  eps = 1e-7
  if abs(F-K)<eps:
    vol = sigma0 / (F**(1-beta))

    correction1 = ((1-beta)**2 * sigma0**2) / (24*F**(2-2*beta))
    correction2 = (rho * beta * alpha * sigma0) / (4*F**(1-beta))
    correction3 = ((2-3*rho**2)/24) * alpha**2

    time_correction = 1 + (correction1 + correction2 + correction3) * T
    return vol * time_correction


  else:
    FK = F*K
    log_FK = np.log(F/K)

    z = (sigma0 / alpha) * (FK**((1-beta)/2)) * log_FK

    sqrt_term = np.sqrt(1 - 2*rho*z + z*z)
    x_z = np.log((sqrt_term + z - rho) / (1 - rho))

    backbone = FK**((1-beta)/2) * (1 + ((1-beta)**2/24)*log_FK**2)

    vol = (sigma0 / backbone) * (z / x_z)

    time_corr = 1 + (
        ((1-beta)**2*sigma0**2)/(24*FK**(1-beta))
        + (rho*beta*sigma0*alpha)/(4*FK**((1-beta)/2))
        + ((2-3*rho**2)/24)*alpha**2
    ) * T

    return vol*time_corr



In [ ]:
class SABRCalibrator:
  def __init__(self):
    self.best_params=None
    self.calibration_error=float("inf")

  def objective_func(self, params, market_data):
    if params.alpha <= 0 or params.beta < 0 or \
      params.beta > 1 or abs(params.rho) >= 1:
      return 1000

    total_error = 0
    total_weight = 0

    for _, row in market_data.iterrows():
      try:
        model_vol = sabr_impl_vol_(
            row["forward"], row["strike"], row["maturity"],
            params.alpha, params.beta, params.rho, row["sigma0"]
        )

        market_vol = row["market_vol"]
        weight = row["weight"]
        error = (model_vol-market_vol)**2

        total_error += error * weight
        total_weight += weight
      except:
        return 1000

    return total_error / total_weight if total_weight<0 else 1000

  def calibrate(self, market_data):
    bounds=[(0.01, 2.0), (0.01, 0.99), (-0.99, 0.99)]
    x0 = [0.2, 0.5, -0.3]
    try:
      result = minimize(
          self.objective_func, x0, args=(market_data),
          bounds=bounds, method="L-BFGS-B"
      )
      if result.success:
        self.best_params = {
            "alpha": result.x[0],
            "beta": result.x[1],
            "rho": result.x[2]
        }

        self.calibration_error=result.fun

        return True

      else:
        return False

    except:
      return False




In [ ]:
def sabr_option_price(
      F:float, K:float, T:float,
      params: SABRParameters,
      option_type:str="call"
    ):
  vol = sabr_impl_vol_(
        F, K, T, params.alpha,
        params.beta, params.rho,
        params.sigma
      )
  if vol <= 0 or T <= 0:
    return max(F-K, 0) if option_type == "call" else max(K-F, 0)

  d1 = (np.log(F/K)+0.5*vol**2*T)/(vol/np.sqrt(T))
  d2 = d1-vol*np.sqrt(T)-K*norm.cdf(d1)
  if option_type=="call":
    return F*norm.cdf(d1)-K*norm.cdf(d2)
  else:
    return K*norm.cdf(-d2)-F*norm.cdf(-d1)

In [ ]:
class SABRModel:
  def __init__(self, params:SABRParameters):
    self.params = params
    self.calibration_results = None
    self.is_calibrated = False
    self.calibration_error = None

  def get_params(self):
    return self.params

  def impl_vol(self, F:float, K:float, T:float):
    return sabr_impl_vol_(F, K, T,
                          self.params.alpha, self.params.beta,
                          self.params.rho, self.params.sigma)

  def option_price(self, F:float, K:float, T:float, option_type:str="call"):
    return sabr_option_price(F, K, T, self.params, option_type)

  def calibrate(self, market_data):
    calibrator = SABRCalibrator()
    success = calibrator.calibrate(market_data)

    if success:
      self.is_calibrated = True
      c_params = calibrator.best_params
      self.best_params = SABRParameters(
          alpha=c_params["alpha"],
          beta=c_params["beta"],
          rho=c_params["rho"],
          sigma=self.params.sigma
      )
      self.calibration_error = calibrator.calibration_error
      return self.is_calibrated

    return self.is_calibrated

  def generate_vol_surface(self, F: float, K_range:tuple[int],
                           T_range:tuple[int], n_strikes:int,
                           n_maturities:int):
    Ks = np.linspace(F*K_range[0], F*K_range[1], n_strikes)
    Ts = np.linspace(T_range[0], T_range[1], n_maturities)

    surface = np.zeros((n_strikes, n_maturities))

    for i, K in enumerate(Ks):
      for j, T in enumerate(Ts):
        try:
          surface[i, j] = self.impl_vol(F, K, T)
        except:
          surface[i, j] = 0

    return Ks, Ts, surface

  def simulate_sabr_dynamics(F0:float, T:int, params:SABRParameters,
                           n_path:int=2000, n_steps:int=100,
                           F_clip:float=0.001, sigma_clip:int=0.01):
    dt = T/n_steps
    F_paths = np.zeros((n_path, n_steps+1))
    sigma_paths = np.zeros((n_path, n_steps+1))
    F_paths[:, 0] = F0
    sigma_paths[:,0] = params.sigma

    for step in range(n_steps):
      W = np.random.normal(size=n_path)
      Z = params.rho*W+np.sqrt(1-params.rho**2)*np.random.normal(size=n_path)

      F_curr = F_paths[:, step]
      sigma_curr = sigma_paths[:, step]

      dF = sigma_curr*(F_curr**params.beta)*np.sqrt(dt)*W

      dsigma = params.alpha*sigma_curr*np.sqrt(dt)*Z

      F_paths[:, step+1] = np.maximum(F_curr+dF, F_clip)
      sigma_paths[:, step+1] = np.maximum(sigma_curr+dsigma, sigma_clip)

    return F_paths, sigma_paths




In [ ]:
params = SABRParameters(alpha=0.3, beta=0.5, rho=-0.4, sigma=0.25)
model = SABRModel(params)
F, K, T = 0.03, 0.035, 0.25
vol = model.impl_vol(F, K, T)
price = model.option_price(F, K, T, option_type="call")
print("impl_vol: ", vol)
print("price: ", price)
model.simulate_sabr_dynamics(F, T, params)